In [55]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from collections import Counter 
from sklearn.utils import shuffle 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df_train=pd.read_csv('./data/train.csv')
df_test=pd.read_csv('./data/test.csv')
df_sub=pd.read_csv('./data/gender_submission.csv')
print(df_train.shape,df_test.shape,df_sub.shape)
print(df_train.dtypes,end='\n')
print(df_test.dtypes,end='\n')
print(df_sub.dtypes)

(891, 12) (418, 11) (418, 2)
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
PassengerId    int64
Survived       int64
dtype: object


In [58]:
print(df_train.head())
print(df_train.info())
print(df_train['Embarked'].unique())

   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null  

In [44]:
df_train.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)
df_test.drop(['Name','Ticket','Cabin'],axis=1,inplace=True)

sex=pd.get_dummies(df_train['Sex'],drop_first=True)
embark=pd.get_dummies(df_train['Embarked'],drop_first=True)
df_train=pd.concat([df_train,sex,embark],axis=1)


In [60]:
print(df_train.dtypes)
df_train.drop(['Sex','Embarked'],axis=1,inplace=True)


PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
dtype: object


KeyError: "['Sex' 'Embarked'] not found in axis"

In [84]:
print(df_train[['Pclass','Survived']].groupby(['Pclass'],as_index=False).mean().sort_values(by='Survived',ascending=False))
print(df_train[['Pclass','Survived']].groupby(['Pclass']).mean().sort_values(by='Pclass'))


   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363
        Survived
Pclass          
1       0.629630
2       0.472826
3       0.242363


In [46]:
sex = pd.get_dummies(df_test['Sex'],drop_first=True)
embark = pd.get_dummies(df_test['Embarked'],drop_first=True)
df_test = pd.concat([df_test,sex,embark],axis=1)

df_test.drop(['Sex','Embarked'],axis=1,inplace=True)


In [47]:
from sklearn.preprocessing import StandardScaler
df_train.fillna(df_train.mean(),inplace=True)
df_test.fillna(df_test.mean(), inplace=True)

Scaler1=StandardScaler()
Scaler2=StandardScaler()

train_columns=df_train.columns
test_columns= df_test.columns 

df_train=pd.DataFrame(Scaler1.fit_transform(df_train))
df_test=pd.DataFrame(Scaler2.fit_transform(df_test))
print(df_train,df_train.dtypes)
print(df_test)
print(train_columns)

            0         1         2         3         4         5         6         7         8         9
0   -1.730108 -0.789272  0.827377 -0.592481  0.432793 -0.473674 -0.502445  0.737695 -0.307562  0.619306
1   -1.726220  1.266990 -1.566107  0.638789  0.432793 -0.473674  0.786845 -1.355574 -0.307562 -1.614710
2   -1.722332  1.266990  0.827377 -0.284663 -0.474545 -0.473674 -0.488854 -1.355574 -0.307562  0.619306
3   -1.718444  1.266990 -1.566107  0.407926  0.432793 -0.473674  0.420730 -1.355574 -0.307562  0.619306
4   -1.714556 -0.789272  0.827377  0.407926 -0.474545 -0.473674 -0.486337  0.737695 -0.307562  0.619306
..        ...       ...       ...       ...       ...       ...       ...       ...       ...       ...
886  1.714556 -0.789272 -0.369365 -0.207709 -0.474545 -0.473674 -0.386671  0.737695 -0.307562  0.619306
887  1.718444  1.266990 -1.566107 -0.823344 -0.474545 -0.473674 -0.044381 -1.355574 -0.307562  0.619306
888  1.722332 -0.789272  0.827377  0.000000  0.432793  2.008933 

In [48]:
df_train.columns=train_columns
df_test.columns=test_columns
features=df_train.iloc[:,2:].columns.tolist()
target=df_train.loc[:,'Survived'].name

X_train=df_train.iloc[:,2:].values
y_train=df_train.loc[:,'Survived'].values

In [49]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.autograd import Variable

In [50]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(8,512 )
        self.fc2=nn.Linear(512,512)
        self.fc3=nn.Linear(512,2)
        self.dropout=nn.Dropout(0.2)
    def forward(self, x):
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        x=F.relu(self.fc2(x))
        x=self.dropout(x)
        x=self.fc3(x)
        return x 
model=Net()
print(model)


Net(
  (fc1): Linear(in_features=8, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [51]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=1e-2)

batch_size=128
n_epochs=500 
batch_no=len(X_train)//batch_size

train_loss=0
train_loss_min=np.Inf
for epoch in range(n_epochs):
    for i in range(batch_no):
        start=i*batch_size
        end=start+batch_size

        x_var=Variable(torch.FloatTensor(X_train[start:end]))
        y_var=Variable(torch.LongTensor(y_train[start:end]))

        optimizer.zero_grad()
        output=model(x_var)
        loss=criterion(output,y_var)
        loss.backward()
        optimizer.step() 

        values,labels=torch.max(output,1)
        num_right=np.sum(labels.data.numpy()==y_train[start:end])
        train_loss+=loss.item()*batch_size
    
    train_loss=train_loss/len(X_train)
    if train_loss<=train_loss_min:
        print(f"validation loss decreased{train_loss_min}->{train_loss}")
        torch.save(model.state_dict(),'model.pt')
        train_loss_min=train_loss

    if epoch%200==0:
        print(f"Epoch:{epoch+1}\t Train_loss:{train_loss}\t Train Accuracy:{num_right/len(y_train[start:end])}")

print('Train End')


validation loss decreasedinf->0.6050595198951318
Epoch:1	 Train_loss:0.6050595198951318	 Train Accuracy:0.0
validation loss decreased0.6050595198951318->0.5826017708260696
validation loss decreased0.5826017708260696->0.564212450391859
validation loss decreased0.564212450391859->0.5508869436955287
validation loss decreased0.5508869436955287->0.5348290153656182
validation loss decreased0.5348290153656182->0.525316635360478
validation loss decreased0.525316635360478->0.5135092541732529
validation loss decreased0.5135092541732529->0.50349576703863
validation loss decreased0.50349576703863->0.49771515643681563
validation loss decreased0.49771515643681563->0.4840710274623309
validation loss decreased0.4840710274623309->0.4783006760800512
validation loss decreased0.4783006760800512->0.465274676580591
validation loss decreased0.465274676580591->0.4619725500927444
validation loss decreased0.4619725500927444->0.45290937897726724
validation loss decreased0.45290937897726724->0.4510903469828194
va

In [52]:
###predictions 

X_test=df_test.iloc[:,1:].values
X_test_var=Variable(torch.FloatTensor(X_test),requires_grad=False)
with torch.no_grad():
    test_result=model(X_test_var)
values,labels= torch.max(test_result,1)
survived=labels.data.numpy()

In [53]:
submission = pd.DataFrame({'PassengerId': df_sub['PassengerId'], 'Survived': survived})
submission.to_csv('submission.csv', index=False)